In [42]:
import cv2
import os
import opennsfw2 as n2
from ultralytics import YOLO

## Extracting frames

In [43]:
import os
import cv2

def extract_frames(video_path, output_folder="Video_frames", interval=0.5):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Delete any existing files in the output folder
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    
    # Load the video
    video_capture = cv2.VideoCapture(video_path)
    
    if not video_capture.isOpened():
        # print("Error: Unable to open video file.")
        return
    
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    
    # print(f"Video FPS: {fps}, Total Frames: {total_frames}, Duration: {duration}s")
    
    frame_count = 0
    saved_frames = 0

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Save a frame every 'interval' seconds
        if frame_count % (fps * interval) == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frames:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            # print(f"Saved {frame_filename}")
            saved_frames += 1
        
        frame_count += 1
        
        # Stop if duration exceeds 30 seconds
        if frame_count > 30 * fps:
            break
    
    video_capture.release()
    # print("Frames extraction completed.")


## Calculation

In [44]:

def percentage(folder_path='cropped_images'):
    # Define the folder containing the images
    folder_path = folder_path
    prob_dict = {}
    # List all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Iterate over all image files
    for image_file in image_files:
        # Construct the full file path
        image_handle = os.path.join(folder_path, image_file)
        
        # Predict NSFW probability for each image
        nsfw_probability = n2.predict_image(image_handle)

        prob_dict[image_handle] = nsfw_probability    
        # Print the result
        # print(f"NSFW probability for {image_file}: {nsfw_probability}")
    filtered_nsfw_dict = {image_name: prob for image_name, prob in prob_dict.items() if prob > 0.2}
    return len(filtered_nsfw_dict)*100/len(prob_dict)

## Cropping main

In [45]:
import os
import cv2
from ultralytics import YOLO

def load_image(image_path):
    """Loads an image from a file path."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image {image_path}. Skipping.")
    return img

def detect_human_boxes(model, image_path):
    """Detects bounding boxes for persons in the image."""
    results = model(image_path, verbose = False)
    detections = results[0].boxes if len(results) > 0 else []
    # Only return bounding boxes for detected persons (class 0)
    human_boxes = [
        (box.xyxy[0].tolist(), box.conf[0].item())
        for box in detections if int(box.cls) == 0  # class 0 is 'person'
    ]
    return human_boxes, detections  # Return both human and all detected boxes

def is_valid_human_box(box, min_area=2000, aspect_ratio_range=(0.2, 2.5)):
    """
    Validates the human bounding box based on area and aspect ratio.
    
    :param box: The bounding box (x1, y1, x2, y2) of the detected person.
    :param min_area: The minimum area of the bounding box to be considered valid.
    :param aspect_ratio_range: The acceptable aspect ratio range for the bounding box.
    :return: True if the bounding box is valid, False otherwise.
    """
    x1, y1, x2, y2 = map(int, box)
    width, height = x2 - x1, y2 - y1
    area = width * height

    # Check if the area is above the minimum threshold
    if area < min_area:
        return False

    # Check if the aspect ratio is within a reasonable range
    aspect_ratio = width / height if height != 0 else 0
    if aspect_ratio < aspect_ratio_range[0] or aspect_ratio > aspect_ratio_range[1]:
        return False

    return True

def crop_and_save_image(img, human_boxes, detections, image_name, output_folder):
    """Crops the largest human person from the image and saves the cropped image."""
    if not human_boxes:
        print(f"No human detected in {image_name}. Skipping.")
        return

    # Find the largest bounding box (human)
    largest_box = None
    largest_area = 0
    for box, confidence in human_boxes:
        if not is_valid_human_box(box):  # Skip invalid boxes
            continue
        x1, y1, x2, y2 = map(int, box)
        width, height = x2 - x1, y2 - y1
        area = width * height
        if area > largest_area:
            largest_area = area
            largest_box = (x1, y1, x2, y2)

    # If there is a valid human box (largest), crop and save the image
    if largest_box:
        x1, y1, x2, y2 = largest_box
        cropped = img[y1:y2, x1:x2]
        output_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_main_person.jpg")
        if cv2.imwrite(output_path, cropped):
            pass
            # print(f"Main person cropped and saved at: {output_path}")
        else:
            pass
            # print(f"Error saving cropped image for {image_name}.")
    else:
        pass
        # print(f"No valid bounding box found for persons in {image_name}. Skipping.")

def crop_main_persons_in_folder(input_folder='Video_frames', output_folder='cropped_images', model_path="yolov8n.pt"):
    """Processes all images in the folder and crops the largest detected person."""
    model = YOLO(model_path)
    os.makedirs(output_folder, exist_ok=True)

    # Delete any existing files in the output folder
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            pass
            # print(f"Processing image: {image_name}")
            
            # Load the image
            img = load_image(image_path)
            if img is None:
                continue

            # Detect persons in the image
            human_boxes, detections = detect_human_boxes(model, image_path)

            # Proceed to cropping only if the largest detected bounding box is a human
            if human_boxes:
                crop_and_save_image(img, human_boxes, detections, image_name, output_folder)
            else:
                print(f"No humans detected in {image_name}. Skipping.")


In [46]:
import cv2
import mediapipe as mp
import os

def detect_and_delete_non_person_images(folder_path='Video_frames'):
    # Initialize Mediapipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)  # For static images
    
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        # Only process image files
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Load the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Unable to load image {image_path}")
                continue
            
            # Convert the image to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Process the image with Mediapipe Pose
            results = pose.process(image_rgb)
            
            # Check if any landmarks are detected
            if results.pose_landmarks:
                pass
                # print(f"Person detected in {filename}")
            else:
                # print(f"No person found in {filename}. Deleting...")
                os.remove(image_path)  # Delete the image


### Extracting frames

In [47]:

# video_file = r"C:\Users\AjayKrishnan\OneDrive - ADA Global\Desktop\coplt\object_detection\cleaned_dataset\nudity\nudity_all_in_one\sample-videos-hul\Video_nudity.mp4"  # Replace with your video file
# output_dir = "Video_frames"

# extract_frames(video_file, output_dir)

### Deleting non-person

In [48]:

# # Example usage
# folder_path = r"C:\Users\AjayKrishnan\OneDrive - ADA Global\Desktop\coplt\object_detection\cleaned_dataset\nudity\nudity_all_in_one\Video_frames"  # Replace with the path to your folder
# detect_and_delete_non_person_images(folder_path)


### Cropping

In [49]:

# # Input and output folders
# input_folder = r"C:\Users\AjayKrishnan\OneDrive - ADA Global\Desktop\coplt\object_detection\cleaned_dataset\nudity\nudity_all_in_one\Video_frames"  # Replace with your input folder containing images
# output_folder = r"cropped_images"  # Replace with your desired output folder

# # Run the function
# crop_main_persons_in_folder(input_folder, output_folder)


### Calculating

In [50]:
# # folder_path = r"C:\Users\AjayKrishnan\OneDrive - ADA Global\Desktop\coplt\object_detection\cleaned_dataset\nudity\nudity_all_in_one\cropped_images"


# folder_path = r"cropped_images"

# ans = percentage(folder_path)
# print("Percetnage: ",ans)

In [51]:
def noods(video_file):
    extract_frames(video_file)
    detect_and_delete_non_person_images()
    crop_main_persons_in_folder()
    ans = percentage()
    return ans

In [52]:
video_path = r"C:\Users\AjayKrishnan\OneDrive - ADA Global\Desktop\coplt\object_detection\cleaned_dataset\nudity\nudity_all_in_one\sample-videos-hul\Video_nudity2.mp4"  # Replace with your video file

In [53]:
res = noods(video_file = video_path)

In [54]:
print(res)

59.375


In [14]:
# Nudity-Positive
# Video_nudity = 29.166666666666668
# Video_nudity2 = 59.375
# sunbath (video from youtube) = 69.23076923076923
# 
# 
# 
# 
# 
# 
# Nudity-Negative
# Tresemme - Disha - Poor.mp4 = 15.384615384615385
# man_walking = 4.0
# Video2 = 28.571428571428573
# skin_care = 0.0
# someothershampoo = 12.068965517241379
# 
# 
# 
# 
# 
# 
# ERRORS
# shampoo = 100              #because ther was only 2 frames
